In [ ]:
%%html
<style>
div.input {
    display:none;
}
li {
    line-height: 1;
}
</style>

In [3]:
%matplotlib inline 

from pandas import Series 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot, transforms
from textwrap import wrap

import re
import ipywidgets as widgets
import PIL as pil

#data = pd.read_csv('1.C.1.csv', sep=';', index_col=[0, 1, 2, 3])
#data.sort_index(inplace=True)

#idx = pd.IndexSlice
#tmp = data.loc[idx[:, 'Psychologie', :, 2018], :]

#print(tmp.index.get_level_values('university').unique())
#display(data.loc['FWF'].xs(2018, level='year'))
#display(data.xs(2018, level='year'))

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)
            
university_short_forms = { 'Universität Wien': 'UWI',
                           'Universität Graz': 'UGR',
                           'Universität Innsbruck': 'UIN',
                           'Universität Salzburg': 'USA',
                           'Universität Klagenfurt': 'UKL',
                           'Universität für Bodenkultur Wien': 'BKW',
                           'Universität für künstlerische und industrielle Gestaltung Linz': 'UGL',
                           'Universität für angewandte Kunst Wien': 'UAW',
                           'Universität Mozarteum Salzburg': 'UMS',
                           'Medizinische Universität Wien': 'MUW',
                           'Medizinische Universität Graz': 'MUG',
                           'Medizinische Universität Innsbruck': 'MUI',
                           'Universität Linz': 'ULI',
                           'Akademie der bildenden Künste Wien': 'ABW',
                           'Technische Universität Wien': 'TUW',
                           'Technische Universität Graz': 'TUG',
                           'Veterinärmedizinische Universität Wien': 'VMW',
                           'Montanuniversität Leoben': 'MUL',
                           'Universität für Weiterbildung Krems': 'UWK',
                           'Wirtschaftsuniversität Wien': 'WUW',
                           'Universität für Musik und darstellende Kunst Wien': 'MKW',
                           'Universität für Musik und darstellende Kunst Graz': 'MKG'
                         }

def shorten(name):
    if name in university_short_forms:
        return university_short_forms[name]
    else:
        return name


class Dimension:
    @classmethod
    def create(cls, dataframe, dimensions):
        dims = dict()
        for dim in dimensions:
            assert dim in dataframe.index.names, "There is no index '" + dim + "'"
            cats = dataframe.index.get_level_values(dim).unique()
            dims[dim] = Dimension(dim, cats.values)
        return dims
    
    def __init__(self, name, categories):
        self.__name = name
        self.__categories = categories   
    @property
    def name (self):
        return self.__name
    @property
    def categories(self):
        return self.__categories  
    def __str__(self):
        return self.name
        
class Datasheet:
    def __init__(self, filename, dimensions, timepoint_name, university_name, value_name):
        dataframe = pd.read_csv(filename, sep=';', index_col=list(range(len(dimensions)+2)))        
        dataframe.columns = [value_name]
        dataframe.sort_index(inplace=True)

        assert timepoint_name in dataframe.index.names, "There is no index '" + timepoint_name + "'"
        assert university_name in dataframe.index.names, "There is no index '" + university_name + "'"
        assert value_name in dataframe, "There is no colum with valid data called '" + value_name + "'"
        
        self.__dataframe = dataframe
        self.__timepoint_idx = timepoint_name
        self.__university_idx = university_name
        self.__value_column = value_name       
        self.__dimensions = Dimension.create(dataframe, dimensions + [university_name, timepoint_name])
        self.__timepoints = dataframe.index.get_level_values(timepoint_name).unique()
        self.__universities = dataframe.index.get_level_values(university_name).unique()
        self.__recent_timepoint = self.__timepoints.max()
                
    @property
    def value_column(self):
        return self.__value_column
    @property
    def university_idx(self):
        return self.__university_idx
    @property
    def timepoint_idx(self):
        return self.__timepoint_idx
    @property
    def dimensions(self):
        return self.__dimensions
    @property
    def most_recent(self):
        return self.__recent_timepoint
    @property
    def universities(self):
        return self.__universities
    
    def only(self, conditions):
        df = self.__dataframe
        if conditions is None:
            return df
        slices = list()
        for dim in self.dimensions.keys():
            if self.dimensions[dim].name in conditions.keys():
                slices.append(conditions[self.dimensions[dim].name])
            else:
                slices.append(slice(None))
        return df.loc[tuple(slices), :]
        
    def analyze(self, description, about, filtering=None):
        return Analysis(self, description, self.dimensions[about], self.only(filtering) \
                            .sum(level = [self.dimensions[about].name, self.university_idx]))


class Analysis:
    def __init__(self, sheet, properties, about, table):
        self.__sheet = sheet
        self.__table = table.unstack(about.name, fill_value=0)
        self.__about = about
        self.__props = properties
        
        self.__sums = table.sum(axis='columns')
        self.__sums.name = sheet.value_column
        self.__totals = table.sum(level=[about.name])[sheet.value_column]
        self.__grand_total = table.sum()
        self.__grand_total.name = 'grand total'
        
        assert self.__grand_total[sheet.value_column] == self.__sums.sum()
        
        self.__structure = Structure(self)
        self.__ranking = Ranking(self)

               
    @property
    def sheet(self):
        return self.__sheet
    @property
    def table(self):
        return self.__table
    @property
    def horizontal_sums(self):
        return self.__sums
    @property
    def sums(self):
        return self.horizontal_sums
    @property
    def vertical_sums(self):
        return self.__totals
    @property
    def totals(self):
        return self.vertical_sums
    @property
    def dim(self):
        return self.__about
    @property
    def description(self):
        return self.__props['description']
    @property
    def about(self):
        return self.__props['about']
    @property
    def scale_description(self):
        return self.__props['scale_description']
    
    def scale(self, value):
        return self.__props['scale'](value)
    def scale_format(self, value):
        return self.__props['scale_format'].format(self.scale(value))
    def format(self, value):
        return self.__props['format'].format(value)
    @property
    def value_format(self):
        return self.__props['format']
    
    
    @property
    def grand_total(self):
        return self.__grand_total
    @property
    def structure(self):
        return self.__structure
    @property
    def ranking(self):
        return self.__ranking
    
    def values(self, university):
        tmp = self.table.loc[university]
        tmp.name = self.sheet.value_column
        return tmp

    
    
class Ranking:
    def __init__(self, analysis):
        self.__analysis = analysis
        self.__rankings = analysis.table.rank(axis='rows', method='max', ascending=False)
        
        tmp = analysis.horizontal_sums.sum(level=self.analysis.sheet.university_idx)
        self.__total_ranking = tmp.rank(method='max', ascending=False).sort_values()
        self.__total_ranking.name = 'rank'
        
    @property
    def analysis(self):
        return self.__analysis
    @property
    def table(self):
        return self.__rankings
    
    def of(self, university, category=None):
        categories = self.__analysis.dim.categories
        retval = None
        if category == None:
            retval = self.__total_ranking.loc[university]
            retval.name = 'ranking'
        else:
            if category in categories:
                retval = self.__rankings[self.analysis.sheet.value_column, category][university].astype(int)
            else:
                retval = self.__rankings.loc[university].astype(int)
        return retval

    def better_than(self, university, category=None):
        if category == None:
            return self.__total_ranking[self.__total_ranking < self.__total_ranking[university]]
        else:
            tmp = self.__rankings[self.analysis.sheet.value_column, category].sort_values()
            return tmp[tmp[university] > tmp]
        
    def best(self, category, top=1):
        tmp = self.__rankings[self.analysis.sheet.value_column, category].sort_values()[0:top]
        return tmp
       
        
            

class Structure:
    def __init__(self, analysis):
        self.__analysis = analysis
        tmp = analysis.sums.sum(level=[analysis.sheet.university_idx])
        self.__table = analysis.table.divide(tmp, axis=0).droplevel(level=0, axis='columns')
        self.__table.name = analysis.sheet.value_column    
        
    @property
    def table(self):
        return self.__table
    @property
    def analysis(self):
        return self.__analysis
    
    def most_important_to(self, university, cutoff=0.9):
        u = self.table.loc[university].sort_values(ascending=False)
        u.name = 'relevance'
        cumsum = u.cumsum()
        cut = (cumsum > cutoff).idxmax()
        return u[:cut]

    def of(self, university):
        tmp = self.table.loc[university].sort_values(ascending=False)
        tmp.name = 'relevance'
        return tmp
    
    def similarities_to(self, university):
        mirror = self.table.swapaxes('columns', 'rows', copy=False)
        uni = mirror[university]
        tmp = pd.DataFrame(data=uni.values).reindex_like(mirror)
        tmp.loc[:, :] = uni[:, None]
        return mirror.combine(tmp, np.minimum)
        
    def most_similar_to(self, university):
        m = self.similarities_to(university)
        result = m.sum(axis='rows')
        result = result.sort_values(ascending=False)[1:]
        result.name = 'similarity'
        return result
        
        
def highlight_places(column, series):
    l = len(series.index)
    if series[column] == 1:
        return ['background-color: green']*l
    elif series[column] == 2 or series[column] == 3:
        return ['background-color: yellow']*l
    else:
        return ['background-color: white']*l
            
class University:
    def __init__(self, name):
        self.__name = name
        self.__analyses = list()
        self.__similarity_matrices = dict()
        self.__similars = dict()
        self.__images = dict()
        self.__similarity_filter = 1/2
        
    def __lt__(self, other):
        return self.name < other.name
        
    @property
    def name(self):
        return self.__name
    @property
    def short(self):
        return shorten(self.__name)
    @property
    def similarity_filter(self):
        return self.__similarity_filter
    @property
    def similars(self):
        return self.__similars
    
    
    def add_analysis(self, analysis):
        a = analysis
        self.__analyses.append(analysis)
        self.__similarity_matrices[a.description] = a.structure.most_similar_to(self.name)
        self.__similars[a.description] = list()
            
        cutoff = 2/3
        series = a.structure.most_important_to(self.name, cutoff)
        series.name = self.name
        df = series.to_frame()
            
        similars = self.__similarity_matrices[a.description]
        tmp = list()
        for i, other in enumerate(similars.index.values):
            if similars[other] < self.similarity_filter:
                if i == 0:
                    break
            else:
                self.__similars[a.description].append(other)
                tmp.append((shorten(other) + '\n{:.0%}'.format(similars[other]), similars[other]))
                series = a.structure.most_important_to(other, 0.99)
                series.name = other
                df = df.merge(series.to_frame(), how='left', on='fos')
        
        df.sort_values(by=self.name, ascending=False)
        identity = df
        rest = 1 - identity.sum()
        df.loc['Andere'] = rest
                
        figure = plt.figure(figsize=(8*1.75, 2*1.75), frameon=False)  
        ax = df.T.plot(kind='barh', ax=plt.gca(), legend=False, stacked=True, fontsize=14)
        ax.invert_yaxis()
        ax.set_xticks([0.5, 2/3, 4/5])#[sim for _, sim in tmp])#[0.5, 0.8])
        ax.set_xticklabels(['{:.1%}'.format(0.5),
                            '{:.1%}'.format(2/3),
                            '{:.1%}'.format(4/5)])#other for other, _ in tmp)#['50%', '80%'])
        
        for i, rect in enumerate(ax.patches):
            fos = i / len(df.columns)
            if rect.get_width() > 0 and i % len(df.columns) == 0:
                 ax.text(rect.get_x() + rect.get_width()*0.6,
                         rect.get_y() - rect.get_height()/2,
                         '\n'.join(wrap(df.index[fos], 20)),
                         rotation=55,
                         ha='left', va='bottom', wrap=True,
                         fontsize=12, color='#000000')
        
        filename = get_valid_filename(self.short + '_' + '_'.join([a.description, 'overview']) + '.png')
        plt.tight_layout()
        plt.savefig(filename, format='png', transparent=True)
        plt.close()
        self.__images[a.description] = filename
        
    def describe(self):
        display(widgets.HTML('<h2 style="page-break-before: always;">' + self.name + '</h2>'))
        
        display(widgets.HTML('<h3>Vergleichbare Universitäten in '
                    + '- und '.join(self.similars.keys()) 
                    + '-Struktur</h3><ul style="line-height: 1;">'))
            
        similars = None
        for similars_list in self.similars.values():
            if similars is None:
                similars = set(similars_list)
            else:
                similars = similars.intersection(set(similars_list))
                
        for s in similars:
            display(widgets.HTML('<li style="margin-bottom: 5px;">' + s + '</li>'))
        if len(similars) == 0:
            display(widgets.HTML('<li><strong>Keine.</strong> Die ' + self.name + ' ist stark spezialisiert.</li>'))
        display(widgets.HTML('</ul>'))
        
        for a in self.__analyses:            
            display(widgets.HTML('<h4>Vergleichbar bei der ' + a.description + '-Struktur</h4><p style="line-height: 1;">'))
            sim_strs = list()
            if len(self.similars[a.description]) == 0:
                sim_strs.append('Die ' + self.name + ' ist im Bereich ' + a.description + ' spezialisiert.<br/>'
                                + '<strong>Keine</strong> andere Universität ähnelt der '
                                + self.name + ' in der ' 
                                + a.description + '-Struktur in auffälliger Weise.')
            
            similars = self.__similarity_matrices[a.description]
            sm = a.structure.similarities_to(self.name)
    
            for i, other in enumerate(self.similars[a.description]):
                if i > 5:
                    break
                sim_str = '<strong>' + other \
                          +  '</strong> (' + '{:.1%}'.format(similars[i]) \
                          + ', wegen &auml;hnlicher interner Relevanz von '
                similarities = sm[other].sort_values(ascending=False)
                sim_str += ', '.join(similarities.head(3).index.values)
                sim_str += ')'
                sim_strs.append(sim_str)
            display(widgets.HTML(', '.join(sim_strs)))
            if len(self.similars[a.description]):
                display(widgets.Image(value=open(self.__images[a.description], 'rb').read(), width=800, height=200))
                
        display(widgets.HTML('</p><small>Universitäten mit einer Ähnlichkeit von weniger als '
                    + '{:.0%}'.format(self.similarity_filter)
                    + ' werden nicht angezeigt.</small>'))
        
        for a in self.__analyses:
            display(widgets.HTML("<h3>" + a.description + "-Struktur der " + self.short + "</h3>"))
            cutoff = 2/3
            df = a.structure.most_important_to(self.name, cutoff)
            identity = df
            rest = 1 - identity.sum()
            display(widgets.HTML('Angezeigt werden ' + a.about + ', die zusammen '
                         + '{:.1%}'.format(identity.sum()) + ' der ' + a.description 
                         + ' an der ' + shorten(self.name) + ' ausmachen. '
                         + 'Alle anderen ' + a.about + ' tragen je weniger als '
                         + '{:.1%}'.format(min(rest, identity.min())) + ' bei.'))
            
            df = df[df > 0].dropna().to_frame()
            df = df.join(a.ranking.of(self.name, 'all')) \
                   .join(a.values(self.name))
            
            best = pd.Series()
            better = pd.Series()
            best.name = 'best'
            better.name = 'better'
            plots = dict()
            
            for cat in df.index.get_level_values(level=a.dim.name):
                best_uni = a.ranking.best(cat).index.values[0]
                if best_uni == self.name:
                    best_uni = '(diese)'
                else: 
                    best_uni += ' (' + a.scale_format(a.values(best_uni)[a.sheet.value_column, cat]) + ')'
                    
                better_than_me = a.ranking.better_than(self.name, cat)
                better_unis = ', '.join(better_than_me.index.values)
                
                my_rank = a.ranking.of(self.name, cat)
                ranks = pd.DataFrame()
                ranks.name = cat
                for top5 in a.ranking.best(cat, top=max(my_rank+1, 4)).index.values:
                    ranks.at[top5, a.sheet.value_column] = a.scale(a.values(top5)[a.sheet.value_column, cat])
                    
                figure = plt.figure(figsize=(6,4))                    
                ax = ranks.plot.bar(ax=plt.gca(), legend=False, width=1, fontsize=20)
                for i, bar in enumerate(ax.patches):
                    if i == my_rank-1:
                        if my_rank == 1:
                            bar.set_facecolor('#10C860')
                        elif 2 <= my_rank <= 3:
                            bar.set_facecolor('#D0D820')
                        else:
                            bar.set_facecolor('#D85010')
                    else:
                        bar.set_facecolor('#C0C0C0')
                ax.set_ylabel(a.scale_description, fontsize=20)
                labels = ax.get_xticklabels()
                for label in labels:
                    label.set_text(shorten(label.get_text()))
                ax.set_xticklabels(labels)
                filename = get_valid_filename(self.short + '_'.join([cat, a.description]) + '.png')
                plt.tight_layout()
                plt.savefig(filename, format='png', transparent=True)
                plt.close()
                plots[cat] = filename
                
                if better_unis == '':
                    better_unis = '(keine)'
                    
                best.at[cat] = best_uni
                better.at[cat] = filename
                
            df = df.join(best, on=a.dim.name).join(better, on=a.dim.name)
            df = df.droplevel(0)
            
            
            df.index.name = a.about
            df.columns = ['Intern', 'Ranking', a.description, 'Beste Universität', 'Universität im Vergleich']
            df = df.sort_values('Intern', ascending=False)
            df['Universität im Vergleich'] = df.index.map(lambda fos: plots[fos])
            
            boxes = list()
            for entry in df.index:
                boxes.append(widgets.VBox(                    
                      (widgets.HTML(('<h4>{entry}</h4><ul style="line-height: 1;">' + 
                                     '<li>{intern:.1%} der {name} an der {uni}</li>' +
                                     '<li>{system:.1%} aller {name} in {entry} ({rank}. Platz)</li></ul>')
                             .format(entry=str(entry),
                                     intern=df['Intern'].loc[entry], 
                                     name=a.description,
                                     system=df[a.description].loc[entry] / a.vertical_sums[str(entry)],
                                     rank=df['Ranking'].loc[entry],
                                     uni=self.short)),
                      widgets.Image(value=open(df['Universität im Vergleich'].loc[entry], "rb").read(), width=6*50, height=4*50),
                      )))
                
            display(widgets.GridBox(children=boxes,
                                    layout=widgets.Layout(grid_template_rows='auto auto auto',
                                                          grid_template_columns='50% 50%')))
                                     
            
            

ds = Datasheet('3.B.1.csv', ['fos'], 'year', 'university', 'funding')
analysis = ds.analyze({'description': 'Publikationen',
                       'about': 'WZW',
                       'format': '{:} Publikationen',
                       'scale_description': 'Publikationen',
                       'scale': lambda x: x,
                       'scale_format': '{:} Publikationen'
                      }, "fos", {'year': ds.most_recent})


ds2 = Datasheet('1.C.1.csv', ['fos', 'funder'], 'year', 'university', 'funding')
analysis2 = ds2.analyze({'description': 'Drittmittel',
                       'about': 'WZW',
                       'format': '{:,} €',
                       'scale_description': 'Millionen Euro',
                       'scale': lambda x: x/1000000,
                       'scale_format': '{:} Mio. €'
                      }, "fos", {'year': ds.most_recent})

universities = list()

for uni_name in ds.universities:
    u = University(uni_name)
    u.add_analysis(analysis)
    u.add_analysis(analysis2)
    universities.append(u)
    
for u in sorted(universities):
    u.describe()
    
    
    

HTML(value='<h2 style="page-break-before: always;">Akademie der bildenden Künste Wien</h2>')

HTML(value='<h3>Vergleichbare Universitäten in Publikationen- und Drittmittel-Struktur</h3><ul style="line-hei…

HTML(value='<li style="margin-bottom: 5px;">Universität für angewandte Kunst Wien</li>')

HTML(value='</ul>')

HTML(value='<h4>Vergleichbar bei der Publikationen-Struktur</h4><p style="line-height: 1;">')

...

HTML(value='Angezeigt werden WZW, die zusammen 85.0% der Publikationen an der WUW ausmachen. Alle anderen WZW …

GridBox(children=(VBox(children=(HTML(value='<h4>Wirtschaftswissenschaften</h4><ul style="line-height: 1;"><li…

HTML(value='<h3>Drittmittel-Struktur der WUW</h3>')

HTML(value='Angezeigt werden WZW, die zusammen 76.9% der Drittmittel an der WUW ausmachen. Alle anderen WZW tr…

GridBox(children=(VBox(children=(HTML(value='<h4>Wirtschaftswissenschaften</h4><ul style="line-height: 1;"><li…